In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch

In [66]:
df = pd.read_csv("./data/pulsar_stars.csv",nrows=4000)

In [67]:
df

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
3995,108.421875,49.875252,0.356194,0.140122,11.224916,43.600214,3.942828,14.564314,0
3996,101.898438,50.420240,0.461758,0.376136,3.197324,23.872246,8.232433,70.314547,0
3997,112.976562,46.050154,0.458158,0.435297,3.989130,22.797107,6.788768,50.696445,0
3998,99.515625,47.455733,0.451288,0.465505,14.775084,48.659888,3.245646,9.195834,0


In [68]:
y_all=df.iloc[:,-1]
X_all = df.iloc[:,0:-1]

In [69]:
X_all.shape

(4000, 8)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all,test_size =0.2, random_state = 0, stratify=y_all)

In [71]:
standardScaler = StandardScaler()
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.transform(X_test)

## torch

In [72]:
X_train[0]
X_train.shape[-1]

8

In [73]:
y_train.values

array([0, 0, 0, ..., 0, 0, 1])

In [74]:
x=torch.tensor(X_train,dtype=float)
y=torch.tensor(y_train.values,dtype=float)

weights=torch.randn((X_train.shape[-1],128),dtype=float,requires_grad=True)
biases=torch.randn(128,dtype=float,requires_grad=True)
weights2=torch.randn((128,1),dtype=float,requires_grad=True)
biases2=torch.randn(1,dtype=float,requires_grad=True)

learning_rate=0.001
losses=[]

for i in range(1000):
    hidden=x.mm(weights)+biases # ax+b
    hidden=torch.relu(hidden)
    predictions=hidden.mm(weights2)+biases2
    
    loss=torch.mean((predictions-y)**2)
    losses.append(loss.data.numpy())

    if i% 100==0:
        print('loss:',loss)
    loss.backward()

    weights.data.add_(-learning_rate*weights.grad.data)
    biases.data.add_(-learning_rate*biases.grad.data)
    weights2.data.add_(-learning_rate*weights2.grad.data)
    biases2.data.add_(-learning_rate*biases2.grad.data)

    weights.grad.data.zero_()
    biases.grad.data.zero_()
    weights2.grad.data.zero_()
    biases2.grad.data.zero_()


loss: tensor(1441.3431, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(6.4783, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(4.0745, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(3.1065, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(2.5945, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(2.2708, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(2.0406, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.8652, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.7208, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.6013, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [75]:
x
y

tensor([0., 0., 0.,  ..., 0., 0., 1.], dtype=torch.float64)